# Install the package

First, you need to install the package from github and import it:

In [1]:
# Install the bulkhours package
!rm -rf bulkhours && git clone https://github.com/guydegnol/bulkhours.git

# import the bulkhours package
import bulkhours

# Generate header links
bulkhours.generate_header_links("examples/1_Simple_Exercice_Automation.ipynb")

Cloning into 'bulkhours'...
remote: Enumerating objects: 5500, done.
remote: Counting objects: 100% (1099/1099), done.
remote: Compressing objects: 100% (425/425), done.
remote: Total 5500 (delta 722), reused 996 (delta 635), pack-reused 4401
Receiving objects: 100% (5500/5500), 41.70 MiB | 23.16 MiB/s, done.
Resolving deltas: 100% (3559/3559), done.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![Open In Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![GitHub](https://badgen.net/badge/icon/Open%20in%20Github?icon=github&label)](https://github.com/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![Open in Visual Studio Code](https://img.shields.io/static/v1?logo=visualstudiocode&label=&message=Open%20in%20Visual%20Studio&labelColor=2c2c32&color=007acc&logoColor=007acc)](https://vscode.dev/github/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![CC-by-nc-sa license](https://badgen.net/badge/icon/CC%20by-nc-sa?label=Licence)](https://creativecommons.org/licenses/by-nc-sa/4.0)


# Create my first exercice


**Exercise**: Using `numpy.exp`, implement the $\sigma(z) = \frac{1}{1 + e^{-z}}$ to make predictions. The expected solution is in the following cell:


In [2]:
import numpy as np

def sigmoid(z):
  s = 1 / (1 + np.exp(-z))
  return s

# Set a unique id for the exercice

To identify an exercice, we need to define an id for each cell. Here the id of the exercice is ```my_first_exercice```:

In [3]:
%%evaluation_cell_id -i my_first_exercice

def sigmoid(z):
  s = 1 / (1 + np.exp(-z))
  return s

for z in [1, 3, 7]:
  print(f"sigmoid({z})={sigmoid(z)}")

# To be able to use your the evaluation tools, you need to identify yourself with the following command:
bulkhours.init_env(email="name@institute.ac")  # To be run in an initisalization cell

# You can also use the following command to get more information:
help(bulkhours.init_env)
    


The execution of the previous cell should have failed. **You need to identify yourself first** :



In [5]:
bulkhours.init_env(email="obiwan.kenobi@jedi.com")

Import BULK Helper cOURSe (version='5.2.24'🚀'):⚠️Database is local (security_level=0). Export your config file if you need persistency.⚠️
(db, subject, virtual_room, nb_id, user) = (free1.json, '', room1, '', obiwan.kenobi@jedi.com✅)


💡**In practice, you should use secured a access with identification checking to avoid security issues**. For example, this is a typical identification used in course:
```
# A secured access to the database for the teacher
bulkhours.init_env(email="dave.gahan@bulkhours.org", database="bkloud@prems/teacher::eNqlVtmSo8gV_ZV-cd...")

# A secured access to a virtual class database for students
bulkhours.init_env(email="dave.gahan@bulkhours.org", database= "bkloud@prems/winterfell_student::eNqVVtmyo8gV_JV-cfRMaLpB7...")
```



# Get the answer from another notebook

Normaly, all the following code should be runned in a teacher notebook.

In demo mode, you can simply switch to the teacher mode calling the
```bulkhours.init_env``` with the ```is_admin=True``` argument/


In [6]:
bulkhours.init_env(email="yoda@jedi.com", is_admin=True)

Import BULK Helper cOURSe (version='5.2.24'🚀', ⚠️ in admin/teacher🎓 mode⚠️):⚠️Database is local (security_level=0). Export your config file if you need persistency.⚠️
(db, subject, virtual_room, nb_id, user) = (free1.json, '', room1, '', yoda@jedi.com🎓)



In real life, this code should be runned from the teacher notebook only

In [8]:
bulkhours.admin.evaluate("my_first_exercice", user="obiwan.kenobi@jedi.com", show_correction=False)

HTML(value="<b><font face='FiraCode Nerd Font' size=4 color='black'>Pas de réponse disponible pour obiwan.keno…

# Automation of the grade


Now we can build a correction for this exercice

In [9]:
%%evaluation_cell_id -i my_first_exercice
import numpy as np

def sigmoid(z):
  s = 1 / (1 + np.exp(-z))
  return s


for z in [1, 3, 7]:
  print(f"sigmoid({z})={sigmoid(z)}")

def student_evaluation_function():
    score = 0
    score += bulkhours.is_equal(sigmoid(np.array([1, 3, 7])), max_score=5)
    score += bulkhours.is_equal(sigmoid(3), teacher.sigmoid(3), max_score=2.5)
    score += bulkhours.is_equal(student.sigmoid(4), teacher.sigmoid(4), max_score=2.5)
    return score

Output()